In [18]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

24/03/06 17:17:27 WARN Utils: Your hostname, 005482.local resolves to a loopback address: 127.0.0.1; using 192.168.23.107 instead (on interface en0)
24/03/06 17:17:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 17:17:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/06 17:17:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/06 17:17:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [8]:
spark.version

'3.5.1'

In [10]:
df = spark.read.parquet("data/raw/fhv/2019/10/*")
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|       264.0|       264.0|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|       264.0|       264.0|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|       264.0|       264.0|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|       264.0|       264.0|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|       264.0|       264.0|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [11]:
df.columns
df = df.withColumnRenamed("pickup_datetime", "pickup_datetime").withColumnRenamed(
    "dropOff_datetime", "dropoff_datetime"
)

In [6]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|       264.0|       264.0|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|       264.0|       264.0|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|       264.0|       264.0|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|       264.0|       264.0|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|       264.0|       264.0|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [12]:
df.count()

1897856

In [13]:
df.repartition(6)

DataFrame[dispatching_base_num: string, pickup_datetime: timestamp_ntz, dropoff_datetime: timestamp_ntz, PUlocationID: double, DOlocationID: double, SR_Flag: int, Affiliated_base_number: string]

In [14]:
df.write.parquet("data/pq/fhv/2019/10/")

In [15]:
df2 = spark.read.parquet("data/pq/fhv/2019/10/*")

In [26]:
df.filter(F.to_date(F.col("pickup_datetime")) == F.to_date(F.lit("2019-10-15"))).count()

62629

In [44]:
df.agg(F.max(F.col("dropoff_datetime") - F.col("pickup_datetime")) / 60.0).show()

+--------------------------------------------------+
|(max((dropoff_datetime - pickup_datetime)) / 60.0)|
+--------------------------------------------------+
|                              INTERVAL '438 07:...|
+--------------------------------------------------+



In [53]:
df.withColumn(
    "hour_diff",
    (F.col("dropoff_datetime") - F.col("pickup_datetime")).cast("long") / (60 * 60),
).agg(F.max(F.col("hour_diff"))).show(truncate=False)

+--------------+
|max(hour_diff)|
+--------------+
|631152.5      |
+--------------+



In [54]:
df.createOrReplaceTempView("fhv")

In [55]:
zones = spark.read.parquet("data/zones/taxi_zone_lookup.parquet")

In [56]:
zones.createOrReplaceTempView("zones")

In [59]:
df_query = spark.sql(
    """
    SELECT 
        f.PULocationID
        ,z.Zone
        ,COUNT(*) cnt
    FROM fhv f
    JOIN zones z
    ON f.PULocationID = z.LocationID
    GROUP BY PULocationID, Zone
    ORDER BY cnt ASC
"""
).show()

+------------+--------------------+---+
|PULocationID|                Zone|cnt|
+------------+--------------------+---+
|         2.0|         Jamaica Bay|  1|
|       105.0|Governor's Island...|  2|
|       111.0| Green-Wood Cemetery|  5|
|        30.0|       Broad Channel|  8|
|       120.0|     Highbridge Park| 14|
|        12.0|        Battery Park| 15|
|       207.0|Saint Michaels Ce...| 23|
|        27.0|Breezy Point/Fort...| 25|
|       154.0|Marine Park/Floyd...| 26|
|         8.0|        Astoria Park| 29|
|       128.0|    Inwood Hill Park| 39|
|       253.0|       Willets Point| 47|
|        96.0|Forest Park/Highl...| 53|
|        34.0|  Brooklyn Navy Yard| 57|
|        59.0|        Crotona Park| 62|
|        58.0|        Country Club| 77|
|        99.0|     Freshkills Park| 89|
|       190.0|       Prospect Park| 98|
|        54.0|     Columbia Street|105|
|       217.0|  South Williamsburg|110|
+------------+--------------------+---+
only showing top 20 rows



In [58]:
zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly